## Import package

In [3]:
# 웹 table 크롤링에 필요한 패키지
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
from selenium.webdriver.common.keys import Keys

# 오류(max url 초과) 해결을 위한 패키지
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

#기본 패키지들
import pyperclip
import pandas as pd
import numpy as np
from numpy import nan as NA
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from collections import Counter
from more_itertools import locate

# For macro rest
import time
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

# pdf 다운로드 창 option 변경 
from selenium.webdriver.chrome.options import Options

# ind_name 매칭
import re
import statistics
import pandas as pd
import numpy as np
from thefuzz import fuzz

import warnings
warnings.filterwarnings(action='ignore')

#change working dir.
import os
os.chdir('E:/루당탕사라')

# 데이터 1: ISMS-P발급현황 데이터

## KISA 정보보호 및 개인정보보호관리체계 인증서 ISMS-P 발급현황 추출
https://isms.kisa.or.kr/main/ispims/issue;jsessionid=8151A7D57431C6C791DBD4DD02D0F0F9/?certificationMode=list

In [4]:
start = time.time()

path = "chromedriver.exe"
wd = webdriver.Chrome(path)
wd.maximize_window() #창 최대화


# 크롤링 준비 ---------------------------------------
num_list = [] ## 인증번호 list (number)
nam_list = [] ## 업체명 list (name)
ran_list = [] ## 인증범위 list (range)
exp_list = [] ## 유효기간 list (expiration)
rem_list = [] ## 취소여부(유지, 취소) list (remain)
# ---------------------------------------------------


for page in range(97): #총 페이지 수
    
    print(f"{page+1} page is crawled...")
    
    # url을 통한 페이지 이동
    url = 'https://isms.kisa.or.kr/main/ispims/issue;jsessionid=B9ACE2CD50B2C5A3FF191D26AC3F3435/?searchCondition=&searchKeyword=&certificationMode=list&crtfYear=&pageIndex=' + str(page+1)
    
    # 사이트로 접속
    wd.get(url)
    wd.implicitly_wait(3)
    
    
    #크롤링 시작
    nums = wd.find_elements_by_xpath('//*[@id="listForm"]/table/tbody/tr/td[1]')
    nams = wd.find_elements_by_xpath('//*[@id="listForm"]/table/tbody/tr/td[2]')
    rans = wd.find_elements_by_xpath('//*[@id="listForm"]/table/tbody/tr/td[3]')
    exps = wd.find_elements_by_xpath('//*[@id="listForm"]/table/tbody/tr/td[4]')
    rems = wd.find_elements_by_xpath('//*[@id="listForm"]/table/tbody/tr/td[5]')
    
    # 텍스트 추출
    new_nums = [i.get_attribute('innerText') for i in nums]
    new_names = [i.get_attribute('innerText') for i in nams]
    new_rans = [i.get_attribute('innerText') for i in rans]
    new_exps = [i.get_attribute('innerText') for i in exps]
    new_rems = [i.get_attribute('innerText') for i in rems]

    # 리스트에 추가 
    num_list.extend(new_nums)
    nam_list.extend(new_names)
    ran_list.extend(new_rans)
    exp_list.extend(new_exps)
    rem_list.extend(new_rems)
    
    time.sleep(3)

time.sleep(10)
wd.quit()

print("time :", time.time() - start)

1 page is crawled...
2 page is crawled...
3 page is crawled...
4 page is crawled...
5 page is crawled...
6 page is crawled...
7 page is crawled...
8 page is crawled...
9 page is crawled...
10 page is crawled...
11 page is crawled...
12 page is crawled...
13 page is crawled...
14 page is crawled...
15 page is crawled...
16 page is crawled...
17 page is crawled...
18 page is crawled...
19 page is crawled...
20 page is crawled...
21 page is crawled...
22 page is crawled...
23 page is crawled...
24 page is crawled...
25 page is crawled...
26 page is crawled...
27 page is crawled...
28 page is crawled...
29 page is crawled...
30 page is crawled...
31 page is crawled...
32 page is crawled...
33 page is crawled...
34 page is crawled...
35 page is crawled...
36 page is crawled...
37 page is crawled...
38 page is crawled...
39 page is crawled...
40 page is crawled...
41 page is crawled...
42 page is crawled...
43 page is crawled...
44 page is crawled...
45 page is crawled...
46 page is crawled.

In [5]:
data = pd.DataFrame({'인증번호':num_list, '업체(기관)명':nam_list, '인증범위':ran_list, '유효기간':exp_list, '취소여부':rem_list})

# NA 확인.
data.isnull().sum()

인증번호       0
업체(기관)명    0
인증범위       0
유효기간       0
취소여부       0
dtype: int64

## ind_name matching
매칭 기준 파일: ind_name_with_CODE.xlsx

In [6]:
# 함수 정의
## 함수를 매번 불러오는 이유는, 파일 상황에 따라 조금씩 코드 수정해야 하는 경우가 빈번하다.
## 따라서 번거롭더라도 함수를 매 시트에서 적용해주고 조건에 따라 코드를 재정의한다.

def match_name(data, col_name):
    
    # 1. 기업명 csv 불러오기 (기준 파일)
    ind_name = pd.read_excel("기업명매칭/ind_name_with_CODE.xlsx", dtype='object')
    
    # 2. Put columns = [CODE, 실제 기업명]
    data.insert(0, 'CODE', -1)
    data.insert(1, 'ind_name', -1)
      ## -1 = 해당 업체명은 우리가 가진 상장기업들에 속하지 않는 이름입니다.
    
    # 3. (주), 주식회사, 본부, 공장 등의 불필요한 단어를 포함한 단어들을 제거한다.
    new_list = []
    remove_word = ['본부', '공장', '사업장', '캠퍼스', '발전소', '발전처']
    
    for name in data[col_name]:
        name = [i for i in re.split('\(주\)|㈜| |\xa0|주식회사', name) if not i==""]
        if len(name) > 1:
            new_list.append(''.join([x for x in name if not any(word in x for word in remove_word)]))
        else:
            new_list.append(name[0])
    
    # 4. 그렇게 생성된 name_list와 우리가 가진 기업명(ind_name)을 매칭하여 기업명을 부여한다.
    for i, name in enumerate(new_list):
    
        ## 매칭 점수 부여 (using fuzz matching: token_sort, token_set_ratio)
        score = [fuzz.token_sort_ratio(name, key) for key in ind_name['trans_name']]
        ## 해당 기업명이 ind_name 중에 하나를 포함하는지의 여부 
        contain_index = [row[0] for row in ind_name.iterrows() if row[1]['trans_name'] in name]
        contain_name = [key for key in ind_name['trans_name'] if key in name]

        if len(contain_name) == 1: ## 포함 기준으로 가져온다.
            data.loc[i, 'CODE'] = ind_name.loc[contain_index[0], 'CODE']
            data.loc[i, 'ind_name'] = ind_name.loc[contain_index[0], 'name']

        elif len(contain_name) == 0: ## 만약 포함되는 단어가 없을 경우, pass. (기업명 없다고 간주)
            continue

        else: ## 만약 여러개 ind_name을 포함할 경우, 그 중 가장 큰 score를 준 ind_name을 부여한다.
            score = [score[x] for x in contain_index]
            data.loc[i, 'CODE'] = ind_name['CODE'][contain_index[score.index(max(score))]]
            data.loc[i, 'ind_name'] = ind_name['name'][contain_index[score.index(max(score))]]

    return(data)

In [7]:
data = match_name(data = data, col_name='업체(기관)명')
data.rename(columns={'CODE':'종목코드', 'ind_name': '회사명'}, inplace=True)
data.to_excel("S crawling/result/ISMS-P발급현황_2021.xlsx", index=None, encoding='cp949')

---

# 데이터 2: 잡플래닛 기업평점

[STEP]
1) 종목코드와 기업명을 불러온다. (기업명은 각 기업에 대해 기존 기업명+변환한 기업명들 = '총 기업명들'을 모두 사용한다.)  
2) 잡플래닛 사이트에 접속하여 기업명을 검색한 URL로 접속한다.   
3) 검색 결과에 나온 잡플래닛_기업명, 잡플래닛_기업평점을 모두 추출한다.  
4) '총 기업명들'과 대조했을 때 가장 높은 점수를 주는 잡플래닛_기업명, 잡플래닛_기업평점을 가져온다.

In [4]:
# STEP 1) 종목코드와 기업명을 불러온다.

ind_name = pd.read_excel('기업명매칭/ind_name_with_CODE.xlsx', dtype='object')

# 평점 들어갈 df 생성
score = pd.DataFrame({'CODE':ind_name['CODE'], 'name':ind_name['name'], 'name_ext':-1, 'score':-1})
score.drop_duplicates(inplace=True)
score.reset_index(inplace=True, drop=True)

In [12]:
# 코드 돌아가는 시간 계산
start_time = time.time()

path = "chromedriver.exe"

wd = webdriver.Chrome(path)
wd.maximize_window() #창 최대화

for i, uniq_name in enumerate(score.name.to_list()):
    co_ext = []
    score_ext = []
    
    # uniq_name에 해당하는 모든 기업명 불러오기
    all_name = ind_name.loc[ind_name['index']==i, 'trans_name']
    
    
    # STEP 2) 잡플래닛 사이트에 접속하여 기업명을 검색한 URL로 접속한다. 
    for name in all_name:
        if '&' in name: #url encoding
            name = name.replace('&', '%26') 
        
        url = 'https://www.jobplanet.co.kr/search?query=' + name + '&category=search_new&search_keyword_hint_id=&_rs_con=seach&_rs_act=keyword_search'

        wd.get(url)
        wd.implicitly_wait(3)
        
        # STEP 3) 검색 결과에 나온 잡플래닛_기업명, 잡플래닛_기업평점을 모두 추출한다.
        try:    
            #모든 기업명 추출
            co_e = wd.find_elements_by_xpath('//*[@id="mainContents"]/div[1]/div/div[2]/div[1]/div/a') #table element로 접근
            co_ext.extend([i.get_attribute("innerText").replace('(주)', '') for i in co_e])
            
            #모든 기업평점 추출
            score_e = wd.find_elements_by_xpath('//*[@id="mainContents"]/div[1]/div/div[2]/div[1]/div/span[3]') #table element로 접근
            score_ext.extend([x.get_attribute("innerText") for x in score_e])
            
        except:
            continue
    
    # STEP 4) '총 기업명들'과 대조했을 때 가장 높은 점수를 주는 잡플래닛_기업명, 잡플래닛_기업평점을 가져온다.
    
    # 만약 co_ext와 score_ext의 길이가 다르거나 길이 0이라면 for문 skip. (데이터 잘못 추출된 것)        
    if not (len(co_ext) == len(score_ext))&(len(co_ext) > 0):
        continue
        
    if len(set(all_name)&set(co_ext)) > 0:
        final_co = list(set(all_name)&set(co_ext))[0]
        final_score = score_ext[co_ext.index(final_co)]
    
    else:
        final_co = co_ext[0]
        final_score = score_ext[0]
    
    #최종 df에 삽입
    score.loc[i, 'name_ext'] = final_co
    score.loc[i, 'score'] = final_score
    
    # 잠시 rest.
    time.sleep(1)


# 코드 돌아간 시간 print.
print(f"Working time : {time.time() - start_time}")

wd.quit()

Working time : 2695.412191390991


In [5]:
score.rename(columns={'CODE':'종목코드', 'name': '회사명'}, inplace=True)
score.to_excel("S crawling/result/잡플래닛평점_2021.xlsx", index=None, encoding="cp949")

---

# 데이터 3: 정보보호 데이터

### 정보보호 공시 종합포털 크롤링
https://isds.kisa.or.kr/publish/list.do?pageNum=1&limit=700


 1) 공시 현황에 접속하여 보이는 표를 전부 크롤링한다. (기업명, 업종, 게시일 컬럼) = 기업의 정보보호 공시 현황 추출   
 2) 기업 매칭을 통해 필요 없는 기업을 제거한다. = 기업명 매칭  
 3) 기업명 매칭된, 필요한 기업만 링크로 접속하여(사이트의 기업명들 클릭하면 접속됨.) 내부의 정보보호 현황을 크롤링한다. = 정보보호 현황 추출

## 1) 기업의 정보보호 공시 현황 추출

In [6]:
## 크롤링 코드 ##

#페이지로 접속, limit=700: 한 페이지 당 최대 기업수 700개로 설정
url = 'https://isds.kisa.or.kr/publish/list.do?pageNum=1&limit=700'
path = "chromedriver.exe"

wd = webdriver.Chrome(path)
wd.maximize_window() #창 최대화

# 사이트로 접속
wd.get(url)
wd.implicitly_wait(3)

# element 추출
names = wd.find_elements_by_xpath('/html/body/div/container/div/div/div[2]/div/table/tbody/tr/td[3]/a') #기업명
tofs = wd.find_elements_by_xpath('/html/body/div/container/div/div/div[2]/div/table/tbody/tr/td[4]/a') #업종
dates = wd.find_elements_by_xpath('/html/body/div/container/div/div/div[2]/div/table/tbody/tr/td[5]') #게시일

# element 내의 텍스트 추출
name_list = [name.get_attribute('innerText') for name in names]
tof_list = [tof.get_attribute('innerText') for tof in tofs]
date_list = [date.get_attribute('innerText') for date in dates]

time.sleep(10) #rest.
wd.quit() #종료.

# 최종 데이터프레임 저장.
info_protect = pd.DataFrame({'기업명':name_list, '업종':tof_list, '게시일':date_list})

## 2) 기업명 매칭

In [8]:
# 함수 정의
## 함수를 매번 불러오는 이유는, 파일 상황에 따라 조금씩 코드 수정해야 하는 경우가 빈번하다.
## 따라서 번거롭더라도 함수를 매 시트에서 적용해주고 조건에 따라 코드를 재정의한다.

def match_name(data, col_name):
    
    # 1. 기업명 csv 불러오기 (기준 파일)
    ind_name = pd.read_excel("기업명매칭/ind_name_with_CODE.xlsx", dtype='object')
    
    # 2. Put columns = [CODE, 실제 기업명]
    data.insert(0, 'CODE', -1)
    data.insert(1, 'ind_name', -1)
      ## -1 = 해당 업체명은 우리가 가진 상장기업들에 속하지 않는 이름입니다.
    
    # 3. (주), 주식회사, 본부, 공장 등의 불필요한 단어를 포함한 단어들을 제거한다.
    new_list = []
    remove_word = ['본부', '공장', '사업장', '캠퍼스', '발전소', '발전처']
    
    for name in data[col_name]:
        name = [i for i in re.split('\(주\)|㈜| |\xa0|주식회사', name) if not i==""]
        if len(name) > 1:
            new_list.append(''.join([x for x in name if not any(word in x for word in remove_word)]))
        else:
            new_list.append(name[0])
    
    # 4. 그렇게 생성된 name_list와 우리가 가진 기업명(ind_name)을 매칭하여 기업명을 부여한다.
    for i, name in enumerate(new_list):
    
        ## 매칭 점수 부여 (using fuzz matching: token_sort, token_set_ratio)
        score = [fuzz.token_sort_ratio(name, key) for key in ind_name['trans_name']]
        ## 해당 기업명이 ind_name 중에 하나를 포함하는지의 여부 
        contain_index = [row[0] for row in ind_name.iterrows() if row[1]['trans_name'] in name]
        contain_name = [key for key in ind_name['trans_name'] if key in name]

        if len(contain_name) == 1: ## 포함 기준으로 가져온다.
            data.loc[i, 'CODE'] = ind_name.loc[contain_index[0], 'CODE']
            data.loc[i, 'ind_name'] = ind_name.loc[contain_index[0], 'name']

        elif len(contain_name) == 0: ## 만약 포함되는 단어가 없을 경우, pass. (기업명 없다고 간주)
            continue

        else: ## 만약 여러개 ind_name을 포함할 경우, 그 중 가장 큰 score를 준 ind_name을 부여한다.
            score = [score[x] for x in contain_index]
            data.loc[i, 'CODE'] = ind_name['CODE'][contain_index[score.index(max(score))]]
            data.loc[i, 'ind_name'] = ind_name['name'][contain_index[score.index(max(score))]]

    return(data)

In [9]:
info_protect_new = match_name(data=info_protect, col_name='기업명')
info_protect_new.to_csv('S crawling/raw_정보보호_2021.csv', index=None, encoding='cp949')

#### 기업명 매칭을 거친 파일은 다시 열어 잘못 매칭된 기업들을 수정하고 엑셀 파일로 저장하였다.
매칭되지 않은 기업 = -1 값은 모두 삭제하고 저장.  
그렇게 수정된 파일: 정보보호_2021.xlsx

## 3) 정보보호 현황 추출

In [20]:
# 수정한 파일 load.
data = pd.read_excel('S crawling/정보보호_2021.xlsx', dtype='object')
data.head()

,CODE,ind_name,기업명,업종,게시일
0,95570,AJ네트웍스,AJ네트웍스 주식회사,임대업; 부동산 제외,2022-06-29
1,265520,AP시스템,에이피시스템(주),기타 기계 및 장비 제조업,2022-06-29
2,282330,BGF리테일,비지에프리테일,도매 및 상품 중개업,2022-06-22
3,079160,CJ CGV,씨제이씨지브이 주식회사,NaN,2022-06-02
4,120,CJ대한통운,씨제이대한통운(주),창고 및 운송관련 서비스업,2022-06-27


In [11]:
result = pd.DataFrame({'기업명':[], '작성된 기업명':[], '정보기술부문 투자액(A)':[], '정보기술부문 투자액(B)':[],
                      '총임직원':[], '정보기술부문 인력(C)':[], '내부인력':[], '외부인력':[], '정보보호부문 전담인력(D) 계':[]})

url = 'https://isds.kisa.or.kr/publish/list.do?pageNum=1&limit=700'
path = "chromedriver.exe"

wd = webdriver.Chrome(path)
wd.maximize_window() #창 최대화


## 크롤링 시작. ##
for i, data_name in enumerate(data['기업명'].tolist()):
    
    # 사이트로 접속
    wd.get(url)
    wd.implicitly_wait(3)
    
    # name_list에서 해당 기업명 찾아서 인덱스 위치 클릭
    names = wd.find_elements_by_xpath('/html/body/div/container/div/div/div[2]/div/table/tbody/tr/td[3]/a')
    if i==0:
        name_list = [name.get_attribute('innerText') for name in names]
    time.sleep(1)

    names[name_list.index(data_name)].click()
    wd.implicitly_wait(3)
    
    
    # 데이터 가져오기
    new_list = [data_name]
    
    ## 기업명, inv A, B 추출
    ### 데이터 값이 없는 경우 error. 이 경우 방지하기 위해 try문을 각각 씌워주었다. (id는 모두 동일)
    try:
        new_list.append(wd.find_element_by_xpath('//*[@id="corpName"]').get_attribute('value'))
    except:
        new_list.append(-1)
    try:
        new_list.append(wd.find_element_by_xpath('//*[@id="investAmountA"]').get_attribute('value'))
    except:
        new_list.append(-1)
    try:
        new_list.append(wd.find_element_by_xpath('//*[@id="investAmountB"]').get_attribute('value'))
    except:
        new_list.append(-1)
    
    ## 인력 데이터 추출
    ### 데이터 값이 없는 경우 error. 이 경우 방지하기 위해 try문을 각각 씌워주었다. (id는 모두 동일)
    try:
        new_list.append(wd.find_element_by_xpath('//*[@id="hrTotal"]').get_attribute('value'))
    except:
        new_list.append(-1)
    try:
        new_list.append(wd.find_element_by_xpath('//*[@id="hrIt"]').get_attribute('value'))
    except:
        new_list.append(-1)
    try:
        new_list.append(wd.find_element_by_xpath('//*[@id="hrItIn"]').get_attribute('value'))
    except:
        new_list.append(-1)
    try:
        new_list.append(wd.find_element_by_xpath('//*[@id="hrItOut"]').get_attribute('value'))
    except:
        new_list.append(-1)
    try:
        new_list.append(wd.find_element_by_xpath('//*[@id="hrItTotal"]').get_attribute('value'))
    except:
        new_list.append(-1)
    
    ## 데이터 프레임에 리스트 데이터 삽입
    result.loc[i] = new_list
       
time.sleep(10)
wd.quit()

### 데이터프레임 merge

In [13]:
# data와 추출된 result를 기업명(크롤링 통해서 추출된 기업명) 기준으로 merge.
merge_DF = pd.merge(data, result, how='inner', on='기업명')
merge_DF.rename(columns={'CODE':'종목코드', 'ind_name': '회사명'}, inplace=True)

merge_DF.to_csv('S crawling/result/정보보호_2021.csv', encoding='cp949')